In [ ]:
pip install 'requests'

In [123]:
import requests
import pandas as pd
import numpy as np
import json

To be able to use the Pôle Emploi API, we need to generate an access token with the client identifier and the secret key provided beforehand, which expires after 24 minutes. 
Once we have the access token, we can query the API and retrieve the job offers in JSON format.
Étape 1: Demander un 'access token' à Pôle Emploi Access Management en spécifiant les API souhaitées et l'id et secret.
    Pôle emploi access management génère et transmet un 'access token' valable pendant 24 minutes.
Étape 2: requêter une API: on fourni l'access token à chaque interrogation d'une API

Étape 1:

In [124]:
# URL pour obtenir le jeton d'accès
token_url = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token?realm=%2Fpartenaire'

# Remplacez 'VOTRE_IDENTIFIANT_CLIENT' et 'VOTRE_CLE_SECRETE' par les valeurs réelles fournies par Pôle Emploi
client_id = 'PAR_poleemploijobsscrapin_4280e6316989cfda7bc759c90500ef347cc95780c6b1d5b67cca3db6c6d373d1'
client_secret = '70d252725959266d147615699b09d63746184cec836d7ff41f1bec6427215373'
# Scopes autorisés pour votre application
scopes = 'api_offresdemploiv2 o2dsoffre'  # Remplacez par les scopes valides pour votre application

# Corps de la requête avec ces paramètres
token_params = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': scopes
}

# En-tête de la requête
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

# Faire la requête pour obtenir le jeton d'accès
# Demander l'access token avec 'post'
response = requests.post(token_url, data=token_params, headers=headers)
access_token=response.json().get('access_token')


Étape 2:

In [125]:
# Requêter l'API avec 'get':
# Soit on spécifie les paramètres dans l'url soit on les mets dans une liste 'params'
#url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search?range=0-2&sort=1&motsCles=data%20engineer'
url = 'https://api.pole-emploi.io/partenaire/offresdemploi/v2/offres/search'

headers = {
    'Authorization': f'Bearer {access_token}'
}

#Boucle pour faire une requête pour chaque mot-clé:
mots_cles_list = ['data engineer', 'data scientist']
responses= []
for mots_cles in mots_cles_list:
    params = {
    'range': '0-149',
    'sort': '1',
    'motsCles': mots_cles,
    }

    # Faire la requête avec les en-têtes d'authentification
    reponse = requests.get(url, headers=headers, params=params)
    responses.append(reponse.json())

responses_json = json.dumps(responses)
responses_list = json.loads(responses_json)
resultats_list = [item['resultats'] for item in responses_list]
resultats_concat = [resultat for sublist in resultats_list for resultat in sublist]
df = pd.DataFrame(resultats_concat)

In [ ]:
lieu_travail_df = pd.json_normalize(df['lieuTravail'])
df = pd.concat([df, lieu_travail_df], axis=1)
df = df.drop(columns=['lieuTravail'])

In [130]:
# Division de la colonne "libelle" en "Dept" et "Ville"
df_Dept_ville = df["libelle"].str.split(" - ", expand=True)
df_Dept_ville.columns = ["Dept", "Ville"]

# Création des nouvelles colonnes "Departement" et "Ville" dans le dataframe
df["departement"] = df_Dept_ville["Dept"]
df["ville"] = df_Dept_ville["Ville"]
df = df.drop(columns=['libelle'])

In [131]:
df['experienceOuiNon'] = np.where(df['experienceExige'] == 'D', 'non', np.where(df['experienceExige'].isin(['E', 'S']), 'oui', None))


In [ ]:
df.columns.values[2] = 'description_offre'
entreprise_df = pd.json_normalize(df['entreprise'])
df = pd.concat([df, entreprise_df], axis=1)
df = df.drop(columns=['entreprise'])

In [136]:
df= df.rename(columns= {'description' : 'entreprise_description', 'nom':'entreprise_nom'})

In [142]:
#Garder que les colonnes nécessaires
colonnes_conserver = ['intitule', 'description_offre', 'dateActualisation', 'typeContrat', 'experienceExige','experienceOuiNon','latitude', 'longitude', 'codePostal', 'commune','departement','ville', 'entreprise_description', 'entreprise_nom', 'salaire', 'secteurActiviteLibelle', 'qualificationLibelle']  
df = df.drop(columns=df.columns.difference(colonnes_conserver))

In [145]:
df.to_csv('Jobs_PoleEmploi.csv', index=False)